## Set up matching DF's

In [580]:
import pandas as pd
# Set pandas to display all columns
pd.set_option('display.max_columns', None)

In [581]:
toto_raw = pd.read_csv('Data/scrapers/Toto/totoAllSports2025-01-21T18:05:01Z.csv', index_col=0).drop_duplicates()
kambi_raw = pd.read_csv('Data/scrapers/unibet/unibetAllSports2025-01-21T16:02:20Z.csv', index_col=0).drop_duplicates()
# Apply the condition and update outcome_label
toto_raw_football = toto_raw[toto_raw['sport'] == 'Voetbal']
kambi_raw_football = kambi_raw[kambi_raw['sport'] == 'FOOTBALL']
# Adjust odds and line
kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
kambi_raw_football['odds'] = kambi_raw_football['odds'] / 1000

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/864075833.py:2: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  kambi_raw = pd.read_csv('Data/scrapers/unibet/unibetAllSports2025-01-21T16:02:20Z.csv', index_col=0).drop_duplicates()
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/864075833.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_raw_football['line'] = kambi_raw_football['line'] / 1000
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/864075833.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [582]:
toto_raw[toto_raw['Event Name'].str.contains('PSV')]

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition
50330,6285726,PSV vs NAC,Resultaat,Gelijkspel,10.00,9.0,1.0,MR,D,Voetbal,Nederland Eredivisie
50331,6285726,PSV vs NAC,Resultaat,PSV,1.11,11.0,100.0,MR,1,Voetbal,Nederland Eredivisie
50332,6285726,PSV vs NAC,Resultaat,NAC,20.00,19.0,1.0,MR,2,Voetbal,Nederland Eredivisie
50333,6285726,PSV vs NAC,Resultaat - Vroege uitbetaling,NAC,13.50,100.0,8.0,MR,2,Voetbal,Nederland Eredivisie
50334,6285726,PSV vs NAC,Resultaat - Vroege uitbetaling,Gelijkspel,8.75,31.0,4.0,MR,D,Voetbal,Nederland Eredivisie
...,...,...,...,...,...,...,...,...,...,...,...
91240,6248641,FK Crvena Zvezda vs PSV,Scoort EN Geeft Assist,L. de Jong scoort EN geeft assist,10.50,19.0,2.0,--,NaN,Voetbal,UEFA Champions League
91241,6248641,FK Crvena Zvezda vs PSV,Scoort EN Geeft Assist,Lang scoort EN geeft assist,15.00,14.0,1.0,--,NaN,Voetbal,UEFA Champions League
91242,6248641,FK Crvena Zvezda vs PSV,Scoort EN Geeft Assist,Saibari scoort EN geeft assist,25.00,24.0,1.0,--,NaN,Voetbal,UEFA Champions League
91243,6248641,FK Crvena Zvezda vs PSV,Scoort EN Geeft Assist,Til scoort EN geeft assist,25.00,24.0,1.0,--,NaN,Voetbal,UEFA Champions League


In [583]:
kambi_raw[kambi_raw['event_name'].str.contains('PSV')]

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name
71388,2486457327,1002275573,Asian Handicap – 1e Helft,Asian Handicap - 1st Half,GOALS,NaN,7,Asian Handicap,Asian Handicap,1021049457,3615938157,PSV Eindhoven,PSV Eindhoven,1470.0,-750.0,PSV Eindhoven,OT_UNTYPED,2025-01-21T12:08:56Z,4/9,-215.0,OPEN,DISABLED,NaN,NaN,PSV Eindhoven vs NAC Breda,FOOTBALL,NaN
71389,2486457327,1002275573,Asian Handicap – 1e Helft,Asian Handicap - 1st Half,GOALS,NaN,7,Asian Handicap,Asian Handicap,1021049457,3615938157,PSV Eindhoven,PSV Eindhoven,1470.0,-750.0,PSV Eindhoven,OT_UNTYPED,2025-01-21T12:08:56Z,4/9,-215.0,OPEN,DISABLED,NaN,NaN,PSV Eindhoven vs NAC Breda,FOOTBALL,Eredivisie
71390,2486457327,1002275573,Asian Handicap – 1e Helft,Asian Handicap - 1st Half,GOALS,NaN,7,Asian Handicap,Asian Handicap,1021049457,3615938160,NAC Breda,NAC Breda,2650.0,750.0,NAC Breda,OT_UNTYPED,2025-01-21T12:08:56Z,33/20,165.0,OPEN,DISABLED,NaN,NaN,PSV Eindhoven vs NAC Breda,FOOTBALL,NaN
71391,2486457327,1002275573,Asian Handicap – 1e Helft,Asian Handicap - 1st Half,GOALS,NaN,7,Asian Handicap,Asian Handicap,1021049457,3615938160,NAC Breda,NAC Breda,2650.0,750.0,NAC Breda,OT_UNTYPED,2025-01-21T12:08:56Z,33/20,165.0,OPEN,DISABLED,NaN,NaN,PSV Eindhoven vs NAC Breda,FOOTBALL,Eredivisie
71392,2486457335,1002275573,Asian Handicap – 1e Helft,Asian Handicap - 1st Half,GOALS,NaN,7,Asian Handicap,Asian Handicap,1021049457,3615938175,PSV Eindhoven,PSV Eindhoven,1670.0,-1000.0,PSV Eindhoven,OT_UNTYPED,2025-01-21T12:08:58Z,4/6,-150.0,OPEN,DISABLED,NaN,NaN,PSV Eindhoven vs NAC Breda,FOOTBALL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158526,2484165508,1002035662,Schoten van Speler op Doel (Afgehandeld Volgen...,Player's shots on target (Settled using Opta d...,NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1021554723,3608209981,Minder dan,Under,1380.0,1500.0,"Pepi, Ricardo",OT_UNDER,2025-01-21T15:17:32Z,4/11,-265.0,OPEN,ENABLED,NaN,NaN,Crvena Zvezda vs PSV Eindhoven,FOOTBALL,NaN
158527,2484165509,1002275572,Asian Handicap,Asian Handicap,GOALS,FULL_TIME,7,Asian Handicap,Asian Handicap,1021554723,3608209982,Crvena Zvezda,Crvena Zvezda,2480.0,250.0,Crvena Zvezda,OT_UNTYPED,2025-01-21T15:17:31Z,29/20,148.0,OPEN,DISABLED,NaN,NaN,Crvena Zvezda vs PSV Eindhoven,FOOTBALL,NaN
158528,2484165509,1002275572,Asian Handicap,Asian Handicap,GOALS,FULL_TIME,7,Asian Handicap,Asian Handicap,1021554723,3608209983,PSV Eindhoven,PSV Eindhoven,1530.0,-250.0,PSV Eindhoven,OT_UNTYPED,2025-01-21T15:17:31Z,13/25,-190.0,OPEN,DISABLED,NaN,NaN,Crvena Zvezda vs PSV Eindhoven,FOOTBALL,NaN
158529,2484165510,1001159730,Scoort met een Kopbal,To score from a header,NaN,FULL_TIME,127,Player Occurrence Line,Player Occurrence Line,1021554723,3608209984,Ja,Yes,18000.0,1000.0,"Duarte, Bruno",OT_YES,2025-01-21T03:14:04Z,17/1,1700.0,OPEN,ENABLED,NaN,NaN,Crvena Zvezda vs PSV Eindhoven,FOOTBALL,NaN


In [584]:
set(kambi_raw_football[kambi_raw_football['criterion_label'].str.contains('Draw No Bet')]['criterion_label'])

{'Draw No Bet', 'Draw No Bet - 1e Helft', 'Draw No Bet - 2e Helft'}

Below is a translation of the abbreviations for the column outcome types into their likely meanings in the context of betting odds:

- `--`: Undefined or No Outcome (e.g., no odds available).
- `AG`: Anytime Goal Scorer (player to score at any time in the match).
- `CS`: Correct Score (predict the exact final score of the event).
- `DC`: Double Chance (cover two outcomes, e.g., Home Win or Draw).
- `DN`: Draw No Bet (stake refunded if the match ends in a draw).
- `FS`: First Scorer (player to score the first goal/point in the event).
- `H1`: First Half Result (outcome at the end of the first half).
- `H2`: Second Half Result (outcome in the second half only).
- `HF`: Half-Time/Full-Time (predict the result at both half-time and full-time).
- `HH`: Head-to-Head (comparison between two participants, e.g., players or teams).
- `HL`: Handicap Line (spread betting; adjust the line to level the playing field).
- `LS`: Last Scorer (player to score the last goal/point in the event).
- `MH`: Match Handicap (spread betting for the entire match).
- `MR`: Match Result (predict the overall winner or draw for the match).
- `OE`: Odd/Even (predict whether the total points/goals scored will be odd or even).
- `WH`: Winning Half (which half will have the higher score).
- `WM`: Winning Margin (predict the margin by which a team/player will win).

In [585]:
# filter down to suitable betting opps
kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_filtered_football
toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

# Remove duplicates in toto_filtered_football based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name']
)
# Remove duplicates in kambi_filtered_football based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label']
)

# Replace 'Sport Club' with 'SC' in the 'event_name' column
kambi_filtered_football['event_name'] = kambi_filtered_football['event_name'].str.replace('Sport Club', 'SC', regex=False)

## Winnaar: Draw no Bet

In [586]:
from rapidfuzz import process, fuzz
import pandas as pd

# Filter kambi_filtered_football based on criterion_label
filtered_kambi_winnaar = kambi_filtered_football[
    kambi_filtered_football['criterion_label'].str.contains('Draw No Bet')
]

# Filter kambi_filtered_football based on criterion_label
filtered_toto_winnaar = toto_filtered_football[
    toto_filtered_football['Market Name'].str.contains('Draw No Bet')
]

# Preprocess strings: Remove extra whitespace and lowercase
def preprocess(text):
    return ' '.join(text.lower().split())

In [587]:
import unicodedata

# Replace '-' with a space and remove accents in the 'names' column
filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)
# Replace '-' with a space and remove accents in the 'names' column
filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

kambi_events = filtered_kambi_winnaar['event_name'].tolist()

# Function to find the best match for 'Event Name' in toto_filtered_football
def find_best_match(event_name):
    # Use a token-based similarity for better handling of substrings
    result = process.extractOne(event_name, kambi_events, scorer=fuzz.token_set_ratio, score_cutoff=95)
    if result is None:  # No match found
        return None
    match, score, _ = result
    return match

# Apply matching function to toto_filtered_football
filtered_toto_winnaar['matched_event'] = filtered_toto_winnaar['Event Name'].apply(find_best_match)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/1713498908.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kambi_winnaar['event_name'] = filtered_kambi_winnaar['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/1713498908.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_toto_winnaar['Event Name'] = filtered_toto_winnaar['Event Name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/1713498908.py:30: SettingW

In [588]:
# from fuzzywuzzy import process, fuzz

# # Function to find the best match and its score
# def find_best_match_with_score(event_name, options):
#     # Use fuzzy matching to find the best match
#     result = process.extractOne(event_name, options, scorer=fuzz.token_set_ratio, score_cutoff=90)
#     if result is None:  # No match found
#         return None, 0  # Return None for match and 0 for score
#     match, score = result
#     return match, score

# # Extract the list of event names from filtered_toto_winnaar
# toto_event_names = filtered_toto_winnaar['Event Name'].tolist()

# # Apply matching function to get both match and score
# filtered_kambi_winnaar[['matched_event', 'match_score']] = filtered_kambi_winnaar['event_name'].apply(
#     lambda x: pd.Series(find_best_match_with_score(x, toto_event_names))
# )

In [589]:
# filtered_kambi_winnaar[~filtered_kambi_winnaar['matched_event'].isna()][['event_name', 'matched_event', 'match_score']].sort_values('match_score', ascending=True)

In [590]:
# Define a transformation function for standardization
def standardize_draw_no_bet(value):
    if '1e Helft' in value:
        return 'Draw No Bet - 1e Helft'
    elif '2e Helft' in value:
        return 'Draw No Bet - 2e Helft'
    elif 'Draw No Bet' in value:
        return 'Draw No Bet'
    else:
        return value  # Keep original if no match

# Apply the transformation to the relevant columns
filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)

# Merge the DataFrames using the standardized column
merged_winnaar = pd.merge(
    filtered_toto_winnaar,
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label'], 
    right_on=['event_name', 'standardized_label'], 
    how='inner'  # Perform an inner join
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/3072039294.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_kambi_winnaar['standardized_label'] = filtered_kambi_winnaar['criterion_label'].apply(standardize_draw_no_bet)
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/3072039294.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_toto_winnaar['standardized_label'] = filtered_toto_winnaar['Market Name'].apply(standardize_draw_no_bet)


In [591]:
set(filtered_toto_winnaar['matched_event'])

{'AC Milan vs Girona',
 'AC Milan vs Inter',
 'AC Milan vs Parma',
 'AS Monaco vs Auxerre',
 'AS Monaco vs Rennes',
 'AZ Alkmaar vs Sparta Rotterdam',
 'Aberdeen vs St Mirren',
 'Adelaide United vs Auckland FC',
 'Airdrieonians vs Dunfermline Athletic',
 'Alianza FC vs Independiente Medellin',
 'Amazulu FC vs Stellenbosch F.C.',
 'America MG vs Pouso Alegre MG',
 'Angers vs Le Havre',
 'Arsenal vs Dinamo Zagreb',
 'Atalanta vs Torino',
 'Athletic Club MG vs Cruzeiro MG',
 'Atletico Madrid vs Bayer Leverkusen',
 'Atletico Mineiro MG vs Democrata GV MG',
 'Atletico San Luis vs Club Necaxa',
 'Auxerre vs Saint Etienne',
 'Ayr United vs Partick Thistle',
 'Barrow vs Grimsby Town',
 'Beerschot vs Union Saint Gilloise',
 'Benfica vs FC Barcelona',
 'Betim MG vs SC Aymores',
 'Blackburn Rovers vs Coventry City',
 'Bologna vs Borussia Dortmund',
 'Bologna vs Como',
 'Boreham Wood vs Hampton & Richmond',
 'Botafogo RJ vs Flamengo RJ',
 'Bradford City vs Walsall',
 'Brescia vs Catanzaro',
 'CF M

In [592]:
filtered_kambi_winnaar

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,standardized_label
246,2485670067,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021067650,3613235346,1,1,1.49,NaN,Auxerre,1,2025-01-20T10:43:30Z,12/25,-205.0,OPEN,DISABLED,NaN,NaN,Auxerre vs Saint Etienne,FOOTBALL,Ligue 1,Draw No Bet - 1e Helft
248,2485670067,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021067650,3613235355,2,2,2.40,NaN,Saint-Étienne,2,2025-01-20T10:43:30Z,7/5,140.0,OPEN,DISABLED,NaN,NaN,Auxerre vs Saint Etienne,FOOTBALL,Ligue 1,Draw No Bet - 1e Helft
250,2485670072,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021067650,3613235385,1,1,1.43,NaN,Auxerre,1,2025-01-20T10:43:30Z,21/50,-235.0,OPEN,DISABLED,NaN,NaN,Auxerre vs Saint Etienne,FOOTBALL,Ligue 1,Draw No Bet
252,2485670072,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021067650,3613235388,2,2,2.80,NaN,Saint-Étienne,2,2025-01-20T10:43:30Z,9/5,180.0,OPEN,DISABLED,NaN,NaN,Auxerre vs Saint Etienne,FOOTBALL,Ligue 1,Draw No Bet
590,2485675227,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021067644,3613244513,1,1,1.38,NaN,AS Monaco,1,2025-01-20T11:26:59Z,4/11,-265.0,OPEN,DISABLED,NaN,NaN,AS Monaco vs Rennes,FOOTBALL,Ligue 1,Draw No Bet - 1e Helft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187432,2487215527,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1022292069,3618573897,2,2,5.10,NaN,Levadiakos,2,2025-01-20T01:29:26Z,4/1,410.0,OPEN,DISABLED,NaN,NaN,PAOK Thessaloniki vs Levadiakos,FOOTBALL,Super League,Draw No Bet - 1e Helft
187585,2487228650,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022292070,3618628894,1,1,1.36,NaN,Olympiakos Piraeus,1,2025-01-19T22:07:54Z,7/20,-278.0,OPEN,DISABLED,NaN,NaN,Olympiakos Piraeus vs Panathinaikos,FOOTBALL,Super League,Draw No Bet
187586,2487228650,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022292070,3618628895,2,2,3.05,NaN,Panathinaikos,2,2025-01-19T22:07:54Z,41/20,205.0,OPEN,DISABLED,NaN,NaN,Olympiakos Piraeus vs Panathinaikos,FOOTBALL,Super League,Draw No Bet
187641,2487228686,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1022292070,3618628964,1,1,1.41,NaN,Olympiakos Piraeus,1,2025-01-19T22:07:55Z,2/5,-245.0,OPEN,DISABLED,NaN,NaN,Olympiakos Piraeus vs Panathinaikos,FOOTBALL,Super League,Draw No Bet - 1e Helft


In [593]:
filtered_toto_winnaar[['Event Name', 'matched_event']]

,Event Name,matched_event
0,Blackburn Rovers vs Coventry City,Blackburn Rovers vs Coventry City
1,Blackburn Rovers vs Coventry City,Blackburn Rovers vs Coventry City
232,Derby County vs Sunderland AFC,Derby County vs Sunderland
233,Derby County vs Sunderland AFC,Derby County vs Sunderland
508,Hull City vs Queens Park Rangers,Hull City vs Queens Park Rangers
...,...,...
130182,SS Monopoli 1966 vs Cavese 1919,None
130197,SS Turris Calcio vs Sorrento Calcio 1945,None
130198,SS Turris Calcio vs Sorrento Calcio 1945,None
130277,Ternana Calcio vs SSD Vis Pesaro 1898,None


In [598]:
filtered_kambi_winnaar


,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,standardized_label
246,2485670067,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021067650,3613235346,1,1,1.49,NaN,Auxerre,1,2025-01-20T10:43:30Z,12/25,-205.0,OPEN,DISABLED,NaN,NaN,Auxerre vs Saint Etienne,FOOTBALL,Ligue 1,Draw No Bet - 1e Helft
248,2485670067,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021067650,3613235355,2,2,2.40,NaN,Saint-Étienne,2,2025-01-20T10:43:30Z,7/5,140.0,OPEN,DISABLED,NaN,NaN,Auxerre vs Saint Etienne,FOOTBALL,Ligue 1,Draw No Bet - 1e Helft
250,2485670072,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021067650,3613235385,1,1,1.43,NaN,Auxerre,1,2025-01-20T10:43:30Z,21/50,-235.0,OPEN,DISABLED,NaN,NaN,Auxerre vs Saint Etienne,FOOTBALL,Ligue 1,Draw No Bet
252,2485670072,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1021067650,3613235388,2,2,2.80,NaN,Saint-Étienne,2,2025-01-20T10:43:30Z,9/5,180.0,OPEN,DISABLED,NaN,NaN,Auxerre vs Saint Etienne,FOOTBALL,Ligue 1,Draw No Bet
590,2485675227,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1021067644,3613244513,1,1,1.38,NaN,AS Monaco,1,2025-01-20T11:26:59Z,4/11,-265.0,OPEN,DISABLED,NaN,NaN,AS Monaco vs Rennes,FOOTBALL,Ligue 1,Draw No Bet - 1e Helft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187432,2487215527,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1022292069,3618573897,2,2,5.10,NaN,Levadiakos,2,2025-01-20T01:29:26Z,4/1,410.0,OPEN,DISABLED,NaN,NaN,PAOK Thessaloniki vs Levadiakos,FOOTBALL,Super League,Draw No Bet - 1e Helft
187585,2487228650,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022292070,3618628894,1,1,1.36,NaN,Olympiakos Piraeus,1,2025-01-19T22:07:54Z,7/20,-278.0,OPEN,DISABLED,NaN,NaN,Olympiakos Piraeus vs Panathinaikos,FOOTBALL,Super League,Draw No Bet
187586,2487228650,1001159666,Draw No Bet,Draw No Bet,GOALS,FULL_TIME,2,Wedstrijd,Match,1022292070,3618628895,2,2,3.05,NaN,Panathinaikos,2,2025-01-19T22:07:54Z,41/20,205.0,OPEN,DISABLED,NaN,NaN,Olympiakos Piraeus vs Panathinaikos,FOOTBALL,Super League,Draw No Bet
187641,2487228686,1001159884,Draw No Bet - 1e Helft,Draw No Bet - 1st Half,GOALS,NaN,2,Wedstrijd,Match,1022292070,3618628964,1,1,1.41,NaN,Olympiakos Piraeus,1,2025-01-19T22:07:55Z,2/5,-245.0,OPEN,DISABLED,NaN,NaN,Olympiakos Piraeus vs Panathinaikos,FOOTBALL,Super League,Draw No Bet - 1e Helft


In [595]:
# # Filter rows in toto_filtered_football where Market Name == 'Winnaar'
# toto_filtered_football = toto_filtered_football[toto_filtered_football['Market Name'].str.contains('Draw No Bet')]

# Perform a left join on the matched_event_name (from toto) and event_name (from kambi)
merged_df_winnaar = filtered_toto_winnaar.merge(
    filtered_kambi_winnaar,
    left_on=['matched_event', 'standardized_label'], 
    right_on=['event_name', 'standardized_label'], 
    how='left'
)

merged_df_winnaar = merged_df_winnaar[(merged_df_winnaar['matched_event'].notnull())&(merged_df_winnaar['Outcome SubType'] != merged_df_winnaar['outcome_label'])]

## Over/Under

In [570]:
# filter down to suitable betting opps
kambi_filtered_football = kambi_raw_football[kambi_raw_football['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered_football = kambi_raw_football
toto_filtered_football = toto_raw_football[toto_raw_football['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

# Remove duplicates in toto_filtered_basketbal based on specified columns
toto_filtered_football = toto_filtered_football.drop_duplicates(
    subset=['Event Name', 'Market Name', 'Outcome Name']
)
# Remove duplicates in kambi_filtered_basketbal based on specified columns
kambi_filtered_football = kambi_filtered_football.drop_duplicates(
    subset=['event_name', 'outcome_label', 'criterion_label']
)

In [571]:
kambi_filtered_football_overunder = kambi_filtered_football[kambi_filtered_football['bet_offer_type_name'].str.contains('Over')]
toto_filtered_football_overunder = toto_filtered_football[(toto_filtered_football['Outcome Name'].str.contains('Over')) | (toto_filtered_football['Outcome Name'].str.contains('Under'))]

In [572]:
kambi_filtered_football_overunder.event_name.unique()

array(['Auxerre vs Saint-Étienne', 'AS Monaco vs Rennes',
       'Strasbourg vs Lille', 'Paris SG vs Reims', 'Le Havre vs Brest',
       'Lens vs Angers', 'Toulouse vs Montpellier', 'Nantes vs Lyon',
       'Nice vs Marseille', 'Montpellier vs Lens', 'Brest vs Paris SG',
       'AS Monaco vs Auxerre', 'Lille vs Saint-Étienne',
       'Toulouse vs Nice', 'Reims vs Nantes', 'Angers vs Le Havre',
       'Rennes vs Strasbourg', 'Marseille vs Lyon',
       'Haguenau vs Dunkerque', 'Metz vs Grenoble', 'Caen vs Guingamp',
       'SC Bastia vs Pau', 'Rodez vs AC Ajaccio', 'Martigues vs Amiens',
       'Paris FC vs Red Star FC', 'Lorient vs Clermont',
       'Arouca vs Moreirense', 'Farense vs Rio Ave',
       'Famalicão vs CF Estrela', 'Sporting Lisbon vs Nacional Madeira',
       'Estoril vs Vitoria Guimarães', 'FC Porto vs Santa Clara',
       'S.C. Braga vs Boavista', 'Monagas SC vs Defensor Sporting',
       'Nacional Asunción (PAR) vs Alianza Lima',
       'Blooming vs El Nacional (ECU)',

In [573]:
import unicodedata

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
kambi_filtered_football_overunder['criterion_english_label'] = kambi_filtered_football_overunder['criterion_english_label'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Event Name'] = toto_filtered_football_overunder['Event Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

# Replace '-' with a space and remove accents in the 'names' column
toto_filtered_football_overunder['Market Name'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: ''.join(
        char for char in unicodedata.normalize('NFKD', x.replace('-', ' '))
        if not unicodedata.combining(char)
    )
)

kambi_events = kambi_filtered_football_overunder['event_name'].tolist()

# Function to find the best match for 'Event Name' in toto_filtered_football
def find_best_match(event_name):
    # Use a token-based similarity for better handling of substrings
    result = process.extractOne(event_name, kambi_events, scorer=fuzz.token_set_ratio, score_cutoff=90)
    if result is None:  # No match found
        return None
    match, score, _ = result
    return match

# Apply matching function to toto_filtered_football
toto_filtered_football_overunder['matched_event'] = toto_filtered_football_overunder['Event Name'].apply(find_best_match)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/1103499553.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['event_name'] = kambi_filtered_football_overunder['event_name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/1103499553.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['criterion_label'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_r

In [574]:
# toto_filtered_football_overunder.to_csv('Data/Testing/toto_filtered_football_overunder.csv')
# kambi_filtered_football_overunder.to_csv('Data/Testing/kambi_filtered_football_overunder.csv')  

In [575]:
# Create 'OverUnderType'
kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
    lambda x: 'Goals' if 'Doelpunten' in x else 'Other'
)

# Create 'OverUnderTime'
def determine_over_under_time(label):
    if '1e helft' in label.lower():
        return '1e Helft'
    elif '2e helft' in label.lower():
        return '2e Helft'
    elif label[-2:].isdigit():  # Check if the last two characters in the label are digits
        return label.split(' - ')[-1]
    else:
        return 'Full Time'

kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)

# Create 'Team1' and 'Team2'
kambi_filtered_football_overunder[['Team1', 'Team2']] = kambi_filtered_football_overunder['event_name'].str.split(' vs ', expand=True)

# Create 'OverUnderType2'
kambi_filtered_football_overunder['OverUnderType2'] = kambi_filtered_football_overunder.apply(
    lambda row: '1' if row['Team1'] in row['criterion_english_label'] else (
        '2' if row['Team2'] in row['criterion_english_label'] else 'Total team 1 and team 2'
    ),
    axis=1
)

# # Create 'line'
# kambi_filtered_football_overunder['line'] = kambi_filtered_football_overunder['line'] / 1000

# # Creat odds
# kambi_filtered_football_overunder['odds'] = kambi_filtered_football_overunder['odds'] / 1000

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/3155310470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderType'] = kambi_filtered_football_overunder['criterion_label'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/3155310470.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kambi_filtered_football_overunder['OverUnderTime'] = kambi_filtered_football_overunder['criterion_label'].apply(determine_over_under_time)
/va

In [576]:
# Create 'OverUnderType'
toto_filtered_football_overunder['OverUnderType'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: 'Goals' if 'Goals' in x else 'Other'
)

# Create 'OverUnderTime'
toto_filtered_football_overunder['OverUnderTime'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: '1e Helft' if '1e helft' in x.lower() else ('2e Helft' if '2e helft' in x.lower() else 'Full Time')
)

# Create 'Team1' and 'Team2'
toto_filtered_football_overunder[['Team1', 'Team2']] = toto_filtered_football_overunder['Event Name'].str.split(' vs ', expand=True)

# Create 'OverUnderType2'
toto_filtered_football_overunder['OverUnderType2'] = toto_filtered_football_overunder.apply(
    lambda row: '1' if row['Team1'] in row['Market Name'] else (
        '2' if row['Team2'] in row['Market Name'] else 'Total team 1 and team 2'
    ),
    axis=1
)

# Create 'line'
toto_filtered_football_overunder['line'] = toto_filtered_football_overunder['Market Name'].apply(
    lambda x: float(x.split('Over/Under ')[-1]) if 'Over/Under ' in x else None
)

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/317070321.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toto_filtered_football_overunder['OverUnderType'] = toto_filtered_football_overunder['Market Name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_58962/317070321.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toto_filtered_football_overunder['OverUnderTime'] = toto_filtered_football_overunder['Market Name'].apply(
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000

In [577]:
# Perform the merge
merged_football_overunder = pd.merge(
    toto_filtered_football_overunder,
    kambi_filtered_football_overunder,
    left_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'matched_event'],
    right_on=['line', 'OverUnderType', 'OverUnderTime', 'OverUnderType2', 'event_name'],
    how='inner'  # Use 'inner' join to get only matching rows; adjust as needed
)
# Keep records with opposite outcomes
merged_football_overunder = merged_football_overunder[merged_football_overunder['outcome_english_label'] != merged_football_overunder['Outcome Name']]

# Display the merged DataFrame
merged_football_overunder.head()

,event_id_x,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,matched_event,OverUnderType,OverUnderTime,Team1_x,Team2_x,OverUnderType2,line,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id_y,outcome_id,outcome_label,outcome_english_label,odds,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport_y,group_name,Team1_y,Team2_y
1,6591063,Blackburn Rovers vs Coventry City,Aantal Goals Over/Under 2.5,Over,2.08,27.0,25.0,HL,1,Voetbal,Engeland Championship,Blackburn Rovers vs Coventry City,Goals,Full Time,Blackburn Rovers,Coventry City,Total team 1 and team 2,2.5,2485977400,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021097614,3614360097,Minder dan,Under,1.70,NaN,OT_UNDER,2025-01-21T15:22:36Z,7/10,-143.0,OPEN,ENABLED,NaN,NaN,Blackburn Rovers vs Coventry City,FOOTBALL,NaN,Blackburn Rovers,Coventry City
2,6591063,Blackburn Rovers vs Coventry City,Aantal Goals Over/Under 2.5,Under,1.73,73.0,100.0,HL,L,Voetbal,Engeland Championship,Blackburn Rovers vs Coventry City,Goals,Full Time,Blackburn Rovers,Coventry City,Total team 1 and team 2,2.5,2485977400,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021097614,3614360090,Meer dan,Over,2.12,NaN,OT_OVER,2025-01-21T15:22:36Z,11/10,112.0,OPEN,ENABLED,NaN,NaN,Blackburn Rovers vs Coventry City,FOOTBALL,NaN,Blackburn Rovers,Coventry City
5,6591064,Derby County vs Sunderland AFC,Aantal Goals Over/Under 1.5,Over,1.39,39.0,100.0,HL,1,Voetbal,Engeland Championship,Derby County vs Sunderland,Goals,Full Time,Derby County,Sunderland AFC,Total team 1 and team 2,1.5,2485982564,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021097609,3614373346,Minder dan,Under,3.15,NaN,OT_UNDER,2025-01-21T15:02:24Z,43/20,215.0,OPEN,ENABLED,NaN,NaN,Derby County vs Sunderland,FOOTBALL,NaN,Derby County,Sunderland
6,6591064,Derby County vs Sunderland AFC,Aantal Goals Over/Under 1.5,Under,2.95,39.0,20.0,HL,L,Voetbal,Engeland Championship,Derby County vs Sunderland,Goals,Full Time,Derby County,Sunderland AFC,Total team 1 and team 2,1.5,2485982564,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021097609,3614373329,Meer dan,Over,1.34,NaN,OT_OVER,2025-01-21T15:02:24Z,1/3,-295.0,OPEN,ENABLED,NaN,NaN,Derby County vs Sunderland,FOOTBALL,NaN,Derby County,Sunderland
9,6608315,Hull City vs Queens Park Rangers,Aantal Goals Over/Under 5.5,Over,12.50,23.0,2.0,HL,1,Voetbal,Engeland Championship,Hull City vs Queens Park Rangers,Goals,Full Time,Hull City,Queens Park Rangers,Total team 1 and team 2,5.5,2485974590,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021097611,3614363982,Minder dan,Under,NaN,NaN,OT_UNDER,2025-01-21T15:10:42Z,NaN,NaN,SUSPENDED,ENABLED,NaN,NaN,Hull City vs Queens Park Rangers,FOOTBALL,NaN,Hull City,Queens Park Rangers


## Match Winnaar & OverUnder and do Arb Analysis

In [ ]:
# Perform the stacked union
total_football = pd.concat([merged_football_overunder, merged_df_winnaar], ignore_index=True, sort=True)
total_football

In [549]:
# Calculate Arbitrage Percentage
total_football['Arbitrage Percentage'] = (1 / total_football['Odds (Decimal)'] + 1 / total_football['odds']) * 100

# Identify Arbitrage Opportunities
total_football['Is Arbitrage'] = total_football['Arbitrage Percentage'] < 100

# Calculate Optimal Stakes if Arbitrage
total_stake = 1000
total_football['Stake A'] = total_football.apply(
    lambda row: (1 / row['Odds (Decimal)'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)
total_football['Stake B'] = total_football.apply(
    lambda row: (1 / row['odds'] / (1 / row['Odds (Decimal)'] + 1 / row['odds']) * total_stake) if row['Is Arbitrage'] else 0,
    axis=1
)

total_football[['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]    

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B
0,Blackburn Rovers vs Coventry City,Aantal Goals Over/Under 2.5,Over,Minder dan,2.08,1.70,106.900452,False,0.0,0.0
1,Blackburn Rovers vs Coventry City,Aantal Goals Over/Under 2.5,Under,Meer dan,1.73,2.12,104.973280,False,0.0,0.0
2,Derby County vs Sunderland AFC,Aantal Goals Over/Under 1.5,Over,Minder dan,1.39,3.15,103.688478,False,0.0,0.0
3,Derby County vs Sunderland AFC,Aantal Goals Over/Under 1.5,Under,Meer dan,2.95,1.34,108.525171,False,0.0,0.0
4,Hull City vs Queens Park Rangers,Aantal Goals Over/Under 5.5,Over,Minder dan,12.50,NaN,NaN,False,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
875,Villa Nova AC MG vs Uberlandia EC MG,Draw No Bet,Uberlandia EC MG,1,2.30,1.46,111.971412,False,0.0,0.0
876,Villa Nova AC MG vs Uberlandia EC MG,1e Helft: Draw No Bet,Villa Nova AC MG,2,1.54,2.28,108.794714,False,0.0,0.0
877,Villa Nova AC MG vs Uberlandia EC MG,1e Helft: Draw No Bet,Uberlandia EC MG,1,2.20,1.55,109.970674,False,0.0,0.0
878,Villa Nova AC MG vs Uberlandia EC MG,2e Helft: Draw No Bet,Villa Nova AC MG,NaN,1.56,NaN,NaN,False,0.0,0.0


In [550]:
merged_football_overunder[merged_football_overunder['Event Name']=='Manchester City WFC vs Manchester United WFC']['criterion_label']

Series([], Name: criterion_label, dtype: object)

In [551]:
total_football[total_football['Is Arbitrage'] == True][['Event Name', 'Market Name', 'Outcome Name', 'outcome_label', 'Odds (Decimal)', 'odds', 'Arbitrage Percentage', 'Is Arbitrage', 'Stake A', 'Stake B']]  

,Event Name,Market Name,Outcome Name,outcome_label,Odds (Decimal),odds,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B
291,APOEL Nicosia vs Omonia Aradippou,Aantal Goals Over/Under 4.5,Under,Meer dan,1.25,6.00,96.666667,True,827.586207,172.413793
362,AC Sparta Praag vs Inter Milan,Aantal Goals Over/Under 3.5,Under,Meer dan,1.49,3.15,98.860126,True,678.879310,321.120690
703,Melbourne Victory vs Sydney FC,Draw No Bet,Sydney FC,1,2.80,1.64,96.689895,True,369.369369,630.630631
779,NAC vs Heracles Almelo,Draw No Bet,Heracles Almelo,1,3.00,3.05,66.120219,True,504.132231,495.867769
782,APOEL Nicosia vs Omonia Aradippou,Draw No Bet,Omonia Aradippou,1,8.00,1.46,80.993151,True,154.334038,845.665962
833,AC Sparta Praag vs Inter Milan,Draw No Bet,AC Sparta Praag,2,6.00,1.58,79.957806,True,208.443272,791.556728


In [552]:
total_football[(total_football['Is Arbitrage'] == True)][['Event Name', 'matched_event', 'Market Name', 'criterion_label', 'Outcome Name', 'outcome_label', 'OverUnderType2',
       'Odds (Decimal)', 'odds', 'Price Numerator', 'Price Denominator',
       'Outcome Type', 'Outcome SubType', 'sport_x', 'competition',
       'OverUnderType', 'OverUnderTime', 'Team1_x', 'Team2_x', 'Team1_y', 'Team2_y', 'Arbitrage Percentage',
       'Is Arbitrage', 'Stake A', 'Stake B']]

,Event Name,matched_event,Market Name,criterion_label,Outcome Name,outcome_label,OverUnderType2,Odds (Decimal),odds,Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport_x,competition,OverUnderType,OverUnderTime,Team1_x,Team2_x,Team1_y,Team2_y,Arbitrage Percentage,Is Arbitrage,Stake A,Stake B
291,APOEL Nicosia vs Omonia Aradippou,Omonia Aradippou vs ALS Omonia,Aantal Goals Over/Under 4.5,Totaal Aantal Doelpunten,Under,Meer dan,Total team 1 and team 2,1.25,6.00,1.0,4.0,HL,L,Voetbal,Cyprus First Division,Goals,Full Time,APOEL Nicosia,Omonia Aradippou,Omonia Aradippou,ALS Omonia,96.666667,True,827.586207,172.413793
362,AC Sparta Praag vs Inter Milan,AC Milan vs Inter,Aantal Goals Over/Under 3.5,Totaal Aantal Doelpunten,Under,Meer dan,Total team 1 and team 2,1.49,3.15,49.0,100.0,HL,L,Voetbal,UEFA Champions League,Goals,Full Time,AC Sparta Praag,Inter Milan,AC Milan,Inter,98.860126,True,678.879310,321.120690
703,Melbourne Victory vs Sydney FC,Melbourne Victory vs Sydney FC,Draw No Bet,Draw No Bet,Sydney FC,1,NaN,2.80,1.64,9.0,5.0,DN,2,Voetbal,Australië W-League,NaN,NaN,NaN,NaN,NaN,NaN,96.689895,True,369.369369,630.630631
779,NAC vs Heracles Almelo,Heracles Almelo vs FC Utrecht,Draw No Bet,Draw No Bet,Heracles Almelo,1,NaN,3.00,3.05,2.0,1.0,DN,2,Voetbal,Nederland Eredivisie,NaN,NaN,NaN,NaN,NaN,NaN,66.120219,True,504.132231,495.867769
782,APOEL Nicosia vs Omonia Aradippou,Omonia Aradippou vs ALS Omonia,Draw No Bet,Draw No Bet,Omonia Aradippou,1,NaN,8.00,1.46,7.0,1.0,DN,2,Voetbal,Cyprus First Division,NaN,NaN,NaN,NaN,NaN,NaN,80.993151,True,154.334038,845.665962
833,AC Sparta Praag vs Inter Milan,AC Milan vs Inter,Draw No Bet,Draw No Bet,AC Sparta Praag,2,NaN,6.00,1.58,5.0,1.0,DN,1,Voetbal,UEFA Champions League,NaN,NaN,NaN,NaN,NaN,NaN,79.957806,True,208.443272,791.556728


In [553]:
merged_football_overunder.to_csv('Data/Testing/merged_football_overunder.csv')

In [29]:
toto_filtered_football_overunder.to_csv('Data/Testing/toto_filtered_football_overunder2.csv')
kambi_filtered_football_overunder.to_csv('Data/Testing/kambi_filtered_football_overunder2.csv')  

In [113]:
kambi_filtered_football_overunder

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,standardized_criterion_label
3993,2482449335,1001159967,Totaal Aantal Doelpunten door Leicester City,Total Goals by Leicester City,NaN,FULL_TIME,6,Over/Onder,Over/Under,1021461108,3602359226,Meer dan,Over,3100.0,1.5,NaN,OT_OVER,2025-01-15T19:47:01Z,21/10,210.0,OPEN,ENABLED,NaN,NaN,Leicester City vs Crystal Palace,FOOTBALL,Premier League,totaal aantal doelpunten door leicester city
3995,2482449335,1001159967,Totaal Aantal Doelpunten door Leicester City,Total Goals by Leicester City,NaN,FULL_TIME,6,Over/Onder,Over/Under,1021461108,3602359227,Minder dan,Under,1340.0,1.5,NaN,OT_UNDER,2025-01-15T19:47:01Z,1/3,-295.0,OPEN,ENABLED,NaN,NaN,Leicester City vs Crystal Palace,FOOTBALL,Premier League,totaal aantal doelpunten door leicester city
3997,2482449341,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021461108,3602359237,Meer dan,Over,19000.0,5.5,NaN,OT_OVER,2025-01-15T19:47:01Z,18/1,1800.0,OPEN,ENABLED,NaN,NaN,Leicester City vs Crystal Palace,FOOTBALL,Premier League,totaal aantal doelpunten
3999,2482449341,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021461108,3602359239,Minder dan,Under,1010.0,5.5,NaN,OT_UNDER,2025-01-15T19:47:01Z,1/100,-10000.0,OPEN,ENABLED,NaN,NaN,Leicester City vs Crystal Palace,FOOTBALL,Premier League,totaal aantal doelpunten
4005,2482449353,1002244276,Asian Totaal,Asian Total,GOALS,FULL_TIME,21,Asian Over/Under,Asian Over/Under,1021461108,3602359263,Meer dan,Over,2140.0,2.5,NaN,OT_OVER,2025-01-15T19:47:01Z,57/50,114.0,OPEN,DISABLED,NaN,NaN,Leicester City vs Crystal Palace,FOOTBALL,Premier League,asian totaal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175114,2486220057,1001159633,Totaal Aantal Doelpunten door ALS Omonia,Total Goals by ALS Omonia,NaN,FULL_TIME,6,Over/Onder,Over/Under,1022536708,3615166761,Minder dan,Under,2880.0,0.5,NaN,OT_UNDER,2025-01-15T19:00:12Z,15/8,188.0,OPEN,ENABLED,NaN,NaN,Omonia Aradippou vs ALS Omonia,FOOTBALL,1ste Divisie,totaal aantal doelpunten door als omonia
175145,2486220062,1001159532,Totaal Aantal Doelpunten - 1e Helft,Total Goals - 1st Half,GOALS,NaN,6,Over/Onder,Over/Under,1022536708,3615166789,Meer dan,Over,1380.0,0.5,NaN,OT_OVER,2025-01-15T19:00:12Z,4/11,-265.0,OPEN,ENABLED,NaN,NaN,Omonia Aradippou vs ALS Omonia,FOOTBALL,1ste Divisie,totaal aantal doelpunten - 1e helft
175146,2486220062,1001159532,Totaal Aantal Doelpunten - 1e Helft,Total Goals - 1st Half,GOALS,NaN,6,Over/Onder,Over/Under,1022536708,3615166795,Minder dan,Under,2750.0,0.5,NaN,OT_UNDER,2025-01-15T19:00:12Z,7/4,175.0,OPEN,ENABLED,NaN,NaN,Omonia Aradippou vs ALS Omonia,FOOTBALL,1ste Divisie,totaal aantal doelpunten - 1e helft
175173,2486220070,1001159967,Totaal Aantal Doelpunten door Omonia Aradippou,Total Goals by Omonia Aradippou,NaN,FULL_TIME,6,Over/Onder,Over/Under,1022536708,3615166841,Meer dan,Over,2350.0,1.5,NaN,OT_OVER,2025-01-15T19:00:12Z,27/20,135.0,OPEN,ENABLED,NaN,NaN,Omonia Aradippou vs ALS Omonia,FOOTBALL,1ste Divisie,totaal aantal doelpunten door omonia aradippou


## Analyze results match

In [35]:
# Calculate the absolute difference between 'odds' and 'Odds (Decimal)'
merged_data['difference'] = abs(merged_data['odds'] - merged_data['Odds (Decimal)'])

# Only keep records where there are 2 outcomes
merged_data = merged_data.groupby(
    ['event_id_toto', 'sport_toto', 'event_id_kambi', 'sport_kambi', 'standard_event_name', 'standard_bet_type']
).filter(lambda x: len(x) == 2)

# Sort by the difference in descending order
top_differences = merged_data.sort_values(by='difference', ascending=False)

# Get the top records with the biggest differences
top_records = top_differences.head(20)

top_records

,event_id_toto,sport_toto,event_id_kambi,sport_kambi,standard_event_name,standard_bet_type,standard_outcome,Line,odds,Odds (Decimal),difference


In [36]:
# Split 'standard_event_name' into two parts
split_names = merged_data['standard_event_name'].str.split(' vs ', expand=True)

# Recode 'standard_outcome'
merged_data['standard_outcome'] = merged_data.apply(
    lambda row: '1' if row['standard_outcome'] == split_names.loc[row.name, 0] 
                else ('2' if row['standard_outcome'] == split_names.loc[row.name, 1] else row['standard_outcome']),
    axis=1
)

merged_data['standard_outcome'] = merged_data['standard_outcome'].astype(str)

ValueError: Columns must be same length as key

In [ ]:
merged_data[merged_data['standard_bet_type'] == 'Jannik Sinner Wint een Set']

,event_id_toto,sport_toto,event_id_kambi,sport_kambi,standard_event_name,standard_bet_type,standard_outcome,Line,odds,Odds (Decimal),difference
2,5944135,Tennis,1022071872,TENNIS,Jannik Sinner vs Taylor Fritz,Jannik Sinner Wint een Set,Ja,NaN,1.04,1.04,0.00
3,5944135,Tennis,1022071872,TENNIS,Jannik Sinner vs Taylor Fritz,Jannik Sinner Wint een Set,Nee,NaN,12.50,8.25,4.25


In [ ]:
## Get odds for single event on 1 record
# Filter groups with exactly two rows
filtered_df = merged_data.groupby(
    ['event_id_toto', 'sport_toto', 'event_id_kambi', 'sport_kambi', 'standard_event_name', 'standard_bet_type']
).filter(lambda x: len(x) == 2)

# Proceed with pivoting only if there are any valid groups left
if not filtered_df.empty:
    # Pivot the filtered DataFrame
    reshaped_df = filtered_df.pivot(
        index=['event_id_toto', 'sport_toto', 'event_id_kambi', 'sport_kambi', 'standard_event_name', 'standard_bet_type'],
        columns='standard_outcome',
        values=['Line', 'odds', 'Odds (Decimal)', 'difference']
    )

    # Flatten the multi-index columns for readability
    reshaped_df.columns = ['_'.join(col).strip() for col in reshaped_df.columns.values]

    # Reset index to turn multi-index into columns
    reshaped_df.reset_index(inplace=True)
else:
    reshaped_df = pd.DataFrame()  # Create an empty DataFrame if no groups qualify

reshaped_df

,event_id_toto,sport_toto,event_id_kambi,sport_kambi,standard_event_name,standard_bet_type,Line_1,Line_2,Line_Ja,Line_Nee,odds_1,odds_2,odds_Ja,odds_Nee,Odds (Decimal)_1,Odds (Decimal)_2,Odds (Decimal)_Ja,Odds (Decimal)_Nee,difference_1,difference_2,difference_Ja,difference_Nee
0,5468089,Mixed Martial Arts,1021633417,MARTIAL_ARTS,Jon Jones vs Stipe Miocic,Wedstrijdnotering,NaN,NaN,NaN,NaN,1.15,5.75,NaN,NaN,1.14,5.25,NaN,NaN,0.01,0.50,NaN,NaN
1,5699912,Voetbal,1020719873,FOOTBALL,San Marino vs Gibraltar,Beide Teams Scoren,NaN,NaN,NaN,NaN,NaN,NaN,2.35,1.53,NaN,NaN,2.33,1.54,NaN,NaN,0.02,0.01
2,5825551,Mixed Martial Arts,1021633419,MARTIAL_ARTS,Bo Nickal vs Paul Craig,Wedstrijdnotering,NaN,NaN,NaN,NaN,1.11,7.00,NaN,NaN,1.08,6.75,NaN,NaN,0.03,0.25,NaN,NaN
3,5848540,Voetbal,1021874423,FOOTBALL,Ecuador vs Bolivia,Beide Teams Scoren,NaN,NaN,NaN,NaN,NaN,NaN,2.08,1.65,NaN,NaN,2.32,1.54,NaN,NaN,0.24,0.11
4,5850619,Voetbal,1021874421,FOOTBALL,Uruguay vs Colombia,Beide Teams Scoren,NaN,NaN,NaN,NaN,NaN,NaN,2.17,1.60,NaN,NaN,2.12,1.64,NaN,NaN,0.05,0.04
5,5850658,Voetbal,1021874447,FOOTBALL,Bolivia vs Paraguay,Beide Teams Scoren,NaN,NaN,NaN,NaN,NaN,NaN,2.14,1.65,NaN,NaN,2.10,1.65,NaN,NaN,0.04,0.00
6,5850661,Voetbal,1021874448,FOOTBALL,Colombia vs Ecuador,Beide Teams Scoren,NaN,NaN,NaN,NaN,NaN,NaN,2.63,1.44,NaN,NaN,2.47,1.48,NaN,NaN,0.16,0.04
7,5865420,Mixed Martial Arts,1022019255,MARTIAL_ARTS,Chris Weidman vs Eryk Anders,Wedstrijdnotering,NaN,NaN,NaN,NaN,2.18,1.68,NaN,NaN,2.10,1.67,NaN,NaN,0.08,0.01,NaN,NaN
8,5880676,Mixed Martial Arts,1022019259,MARTIAL_ARTS,Jonathan Martinez vs Marcus McGhee,Wedstrijdnotering,NaN,NaN,NaN,NaN,2.10,1.74,NaN,NaN,2.10,1.67,NaN,NaN,0.00,0.07,NaN,NaN
9,5885309,Mixed Martial Arts,1022019261,MARTIAL_ARTS,Jim Miller vs Damon Jackson,Wedstrijdnotering,NaN,NaN,NaN,NaN,2.23,1.65,NaN,NaN,2.25,1.59,NaN,NaN,0.02,0.06,NaN,NaN


In [ ]:
reshaped_df

Line  \
standard_outcome                                                                                                                 Adam Mould   
event_id_toto sport_toto         event_id_kambi sport_kambi  standard_event_name                   standard_bet_type                          
5468088       Mixed Martial Arts 1021633418     MARTIAL_ARTS Charles Oliveira vs Michael Chandler  Wedstrijdnotering                    NaN   
5468089       Mixed Martial Arts 1021633417     MARTIAL_ARTS Jon Jones vs Stipe Miocic             Wedstrijdnotering                    NaN   
5699912       Voetbal            1020719873     FOOTBALL     San Marino vs Gibraltar               Beide Teams Scoren                   NaN   
5825551       Mixed Martial Arts 1021633419     MARTIAL_ARTS Bo Nickal vs Paul Craig               Wedstrijdnotering                    NaN   
5848540       Voetbal            1021874423     FOOTBALL     Ecuador vs Bolivia                    Beide Teams Scoren                   NaN   
5850619       Voetbal            1021874421     FOOTBALL     Uruguay vs Colombia                   Beide Teams Scoren                   NaN   
5850658       Voetbal            1021874447     FOOTBALL     Bolivia vs Paraguay                   Beide Teams Scoren                   NaN   
5850661       Voetbal            1021874448     FOOTBALL     Colombia vs Ecuador                   Beide Teams Scoren                   NaN   
5865420       Mixed Martial Arts 1022019255     MARTIAL_ARTS Chris Weidman vs Eryk Anders          Wedstrijdnotering                    NaN   
5880676       Mixed Martial Arts 1022019259     MARTIAL_ARTS Jonathan Martinez vs Marcus McGhee    Wedstrijdnotering                    NaN   
5885309       Mixed Martial Arts 1022019261     MARTIAL_ARTS Jim Miller vs Damon Jackson           Wedstrijdnotering                    NaN   
5887010       Mixed Martial Arts 1022019258     MARTIAL_ARTS Mickey Gall vs Ramiz Brahimaj         Wedstrijdnotering                    NaN   
5896181       Mixed Martial Arts 1022019257     MARTIAL_ARTS Mauricio Ruffy vs James Llontop       Wedstrijdnotering                    NaN   
5905931       Voetbal            1021999829     FOOTBALL     Angola vs Ghana                       Beide Teams Scoren                   NaN   
5905936       Voetbal            1021983458     FOOTBALL     Chad vs Sierra Leone                  Beide Teams Scoren                   NaN   
5905938       Voetbal            1021983457     FOOTBALL     Liberia vs Togo                       Beide Teams Scoren                   NaN   
5915669       Voetbal            1021991853     FOOTBALL     Benin vs Nigeria                      Beide Teams Scoren                   NaN   
5915694       Voetbal            1022007570     FOOTBALL     Ethiopia vs Tanzania                  Beide Teams Scoren                   NaN   
5915715       Voetbal            1022007578     FOOTBALL     Guinea vs DR Congo                    Beide Teams Scoren                   NaN   
5915736       Voetbal            1021999832     FOOTBALL     Eswatini vs Guinea-Bissau             Beide Teams Scoren                   NaN   
5915902       Voetbal            1021999831     FOOTBALL     Mozambique vs Mali                    Beide Teams Scoren                   NaN   
5915941       Voetbal            1021991850     FOOTBALL     Burkina Faso vs Senegal               Beide Teams Scoren                   NaN   
5915943       Voetbal            1021991848     FOOTBALL     Burundi vs Malawi                     Beide Teams Scoren                   NaN   
5922618       Darten             1022055773     DARTS        Richie Burnett vs Adam Mould          Wedstrijd                            NaN   
5922623       Darten             1022055775     DARTS        Tommy Morris vs Johann Brouwer        Wedstrijd                            NaN   
5922624       Darten             1022055765     DARTS        Tommy Morris vs Richie Burnett        Wedstrijd                       

In [ ]:
matched_events

{'Daniil Medvedev vs De Alex Minaur': 'Daniil Medvedev vs Alex De Minaur',
 'Jannik Sinner vs Taylor Fritz': 'Jannik Sinner vs Taylor Fritz',
 'Carlos Alcaraz vs Andrey Rublev': 'Carlos Alcaraz vs Andrey Rublev',
 'Alexander Zverev vs Casper Ruud': 'Alexander Zverev vs Casper Ruud',
 'Reece Robinson vs Sebastian Białecki': 'Reece Robinson vs Sebastian Bialecki',
 'Richie Burnett vs Adam Mould': 'Richie Burnett vs Adam Mould',
 'Tommy Morris vs Johann Brouwer': 'Tommy Morris vs Johann Brouwer',
 'Adam Mould vs Reece Robinson': 'Adam Mould vs Reece Robinson',
 'Sebastian Białecki vs Tommy Morris': 'Sebastian Bialecki vs Tommy Morris',
 'Johann Brouwer vs Richie Burnett': 'Johann Brouwer vs Richie Burnett',
 'Sebastian Białecki vs Adam Mould': 'Sebastian Bialecki vs Adam Mould',
 'Reece Robinson vs Johann Brouwer': 'Reece Robinson vs Johann Brouwer',
 'Tommy Morris vs Richie Burnett': 'Tommy Morris vs Richie Burnett',
 'Johann Brouwer vs Sebastian Białecki': 'Johann Brouwer vs Sebastian B

In [ ]:
# event name is key
# event keys: Wedstrijd = Wedstrijd, outcome = 1 vs 2
#             Over/Under criterion_label + line : 'Totaal Aantal Gewonnen Games door Alcaraz, Carlos' + 125000.0 vs Market Name: 'Carlos Alcaraz Aantal Games - Over/Under 12.5'
set(kambi_filtered[kambi_filtered['sport'] == 'TENNIS'][['criterion_label',
       'criterion_english_label', 'occurrence_type', 'lifetime',
       'bet_offer_type_id', 'bet_offer_type_name',
       'bet_offer_type_english_name', 'event_id', 'outcome_id',
       'outcome_label', 'outcome_english_label', 'odds', 'line', 'participant',
       'type', 'status',
       'cash_out_status', 'home_score', 'away_score', 'event_name']].criterion_label)

{'Alex Minaur Wint een Set',
 'Alexander Zverev Wint een Set',
 'Andrey Rublev Wint een Set',
 'Carlos Alcaraz Wint een Set',
 'Casper Ruud Wint een Set',
 'Correcte Score - Set 1',
 'Daniil Medvedev Wint een Set',
 'Game Handicap',
 'Game Handicap - Set 1',
 'Jannik Sinner Wint een Set',
 'Meeste aces',
 'Set Handicap',
 'Setwedden',
 'Taylor Fritz Wint een Set',
 'Totaal Aantal Games',
 'Totaal Aantal Games - Set 1',
 'Totaal Aantal Games - Set 2',
 'Totaal Aantal Gewonnen Games door Alcaraz, Carlos',
 'Totaal Aantal Gewonnen Games door De Minaur, Alex',
 'Totaal Aantal Gewonnen Games door Fritz, Taylor',
 'Totaal Aantal Gewonnen Games door Medvedev, Daniil',
 'Totaal Aantal Gewonnen Games door Rublev, Andrey',
 'Totaal Aantal Gewonnen Games door Ruud, Casper',
 'Totaal Aantal Gewonnen Games door Sinner, Jannik',
 'Totaal Aantal Gewonnen Games door Zverev, Alexander',
 'Totaal Aantal Sets',
 'Totaal Aantal Tiebreaks',
 'Totaal aantal minuten',
 'Totaal aantal servicebreaks',
 'Wedstr

In [ ]:
kambi_filtered[kambi_filtered['criterion_label']!='Wedstrijd'][['criterion_label',
       'criterion_english_label', 'occurrence_type', 'lifetime',
       'bet_offer_type_id', 'bet_offer_type_name',
       'bet_offer_type_english_name', 'event_id', 'outcome_id',
       'outcome_label', 'outcome_english_label', 'odds', 'line', 'participant',
       'type', 'status',
       'cash_out_status', 'home_score', 'away_score', 'event_name']].tail(60)

,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,status,cash_out_status,home_score,away_score,event_name
81408,Dubbele Kans - 1e Helft,Double Chance - 1st Half,GOALS,NaN,12,Dubbele kans,Double Chance,1021152946,3565547609,12,12,1640.0,NaN,NaN,OT_ONE_OR_TWO,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81409,Dubbele Kans - 1e Helft,Double Chance - 1st Half,GOALS,NaN,12,Dubbele kans,Double Chance,1021152946,3565547615,X2,X2,1230.0,NaN,NaN,OT_CROSS_OR_TWO,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81410,3-Way Handicap,3-Way Handicap,GOALS,FULL_TIME,11,3-Way handicap,3-Way Handicap,1021152946,3565547608,1,1,6750.0,-1000.0,Port Vale,1,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81411,3-Way Handicap,3-Way Handicap,GOALS,FULL_TIME,11,3-Way handicap,3-Way Handicap,1021152946,3565547614,X,X,4700.0,-1000.0,NaN,OT_CROSS,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81412,3-Way Handicap,3-Way Handicap,GOALS,FULL_TIME,11,3-Way handicap,3-Way Handicap,1021152946,3565547622,2,2,1290.0,-1000.0,Wrexham,2,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81413,Beide Teams Scoren in Beide Helften,Both Teams to Score in Both Halves,NaN,NaN,18,Ja/Nee,Yes/No,1021152946,3565547638,Ja,Yes,15000.0,NaN,NaN,OT_YES,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81414,Beide Teams Scoren in Beide Helften,Both Teams to Score in Both Halves,NaN,NaN,18,Ja/Nee,Yes/No,1021152946,3565547667,Nee,No,1020.0,NaN,NaN,OT_NO,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81415,Correcte Score,Correct Score,GOALS,FULL_TIME,3,Correcte Score,Correct Score,1021152946,3565547619,0-0,0-0,9500.0,NaN,NaN,OT_UNTYPED,OPEN,ENABLED,0.0,0.0,Port Vale vs Wrexham
81416,Correcte Score,Correct Score,GOALS,FULL_TIME,3,Correcte Score,Correct Score,1021152946,3565547627,0-1,0-1,7500.0,NaN,NaN,OT_UNTYPED,OPEN,ENABLED,0.0,1.0,Port Vale vs Wrexham
81417,Correcte Score,Correct Score,GOALS,FULL_TIME,3,Correcte Score,Correct Score,1021152946,3565547635,0-2,0-2,10500.0,NaN,NaN,OT_UNTYPED,OPEN,ENABLED,0.0,2.0,Port Vale vs Wrexham


In [ ]:
toto_filtered[(toto_filtered['sport']=='Tennis') & (toto_filtered['event_id']==5943214)].drop_duplicates().head(50)

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition
0,5943214,Daniil Medvedev vs Alex De Minaur,Alex De Minaur Wint een Set,Ja,1.35,7,20,--,NaN,Tennis,"ATP Finals, Enkelspel"
3,5943214,Daniil Medvedev vs Alex De Minaur,Alex De Minaur Wint een Set,Nee,2.95,39,20,--,NaN,Tennis,"ATP Finals, Enkelspel"
6,5943214,Daniil Medvedev vs Alex De Minaur,Wedstrijd,Alex De Minaur,1.93,93,100,HH,2,Tennis,"ATP Finals, Enkelspel"
9,5943214,Daniil Medvedev vs Alex De Minaur,Wedstrijd,Daniil Medvedev,1.92,23,25,HH,1,Tennis,"ATP Finals, Enkelspel"
12,5943214,Daniil Medvedev vs Alex De Minaur,Aantal Games - Odd/Even,Odd,1.87,87,100,OE,1,Tennis,"ATP Finals, Enkelspel"
15,5943214,Daniil Medvedev vs Alex De Minaur,Aantal Games - Odd/Even,Even,1.87,87,100,OE,2,Tennis,"ATP Finals, Enkelspel"
18,5943214,Daniil Medvedev vs Alex De Minaur,Set - Handicap 1.5,Alex De Minaur,2.95,39,20,WH,2,Tennis,"ATP Finals, Enkelspel"
21,5943214,Daniil Medvedev vs Alex De Minaur,Set - Handicap 1.5,Daniil Medvedev,1.35,7,20,WH,1,Tennis,"ATP Finals, Enkelspel"
24,5943214,Daniil Medvedev vs Alex De Minaur,Exact Sets,2 Sets,1.59,59,100,--,NaN,Tennis,"ATP Finals, Enkelspel"
27,5943214,Daniil Medvedev vs Alex De Minaur,Exact Sets,3 Sets,2.27,127,100,--,NaN,Tennis,"ATP Finals, Enkelspel"


In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz, process

# Function to normalize names
def normalize_name(name):
    name = re.sub(r'\b(de|het|een)\b', '', name, flags=re.IGNORECASE)  # Remove articles
    name = re.sub(r'\s+', ' ', name).strip()  # Remove extra spaces
    return name

# Function to match names using fuzzy matching
def match_names(kambi_names, toto_names):
    normalized_toto = {name: normalize_name(name) for name in toto_names}
    normalized_kambi = {name: normalize_name(name) for name in kambi_names}
    matched_names = {}
    for kambi_name, normalized_kambi_name in normalized_kambi.items():
        match, score = process.extractOne(normalized_kambi_name, normalized_toto.values(), scorer=fuzz.token_sort_ratio)
        if score > 85:  # Adjust threshold
            matched_names[kambi_name] = [k for k, v in normalized_toto.items() if v == match][0]
        else:
            matched_names[kambi_name] = kambi_name
    return matched_names

# Normalize and map names in Kambi and Toto
kambi_names = kambi_filtered['bet_offer_type_name'].unique()
toto_names = toto_filtered['Market Name'].unique()
matched_names = match_names(kambi_names, toto_names)
kambi_filtered['normalized_bet_offer_type'] = kambi_filtered['bet_offer_type_name'].map(matched_names)
toto_filtered['normalized_market_name'] = toto_filtered['Market Name'].apply(normalize_name)

# Merge DataFrames using normalized columns
merged_data = pd.merge(
    kambi_filtered,
    toto_filtered,
    left_on=['standard_event_name', 'normalized_bet_offer_type', 'standard_outcome', 'Line'],
    right_on=['standard_event_name', 'normalized_market_name', 'standard_outcome', 'Line'],
    suffixes=('_kambi', '_toto')
)

# Output merged results
print(merged_data[['event_name', 'Market Name', 'criterion_label', 'outcome_label', 'Line']].drop_duplicates())

                          event_name Market Name             criterion_label  \
0      Jannik Sinner vs Taylor Fritz   Wedstrijd           Wedstrijdnotering   
1      Jannik Sinner vs Taylor Fritz   Wedstrijd           Wedstrijdnotering   
2    Carlos Alcaraz vs Andrey Rublev   Wedstrijd           Wedstrijdnotering   
3    Carlos Alcaraz vs Andrey Rublev   Wedstrijd           Wedstrijdnotering   
4    Alexander Zverev vs Casper Ruud   Wedstrijd           Wedstrijdnotering   
..                               ...         ...                         ...   
236       Mark Selby vs Shaun Murphy   Wedstrijd                     Frame 1   
238       Mark Selby vs Shaun Murphy   Wedstrijd       Meeste Century Breaks   
240       Mark Selby vs Shaun Murphy   Wedstrijd       Meeste Century Breaks   
242       Mark Selby vs Shaun Murphy   Wedstrijd  Meeste Half-Century Breaks   
244       Mark Selby vs Shaun Murphy   Wedstrijd  Meeste Half-Century Breaks   

         outcome_label  Line  
0       

In [ ]:
merged_data[['event_name', 'Market Name', 'criterion_label', 'outcome_label', 'Line', 'odds', 'Odds (Decimal)','sport_toto']].drop_duplicates()

,event_name,Market Name,criterion_label,outcome_label,Line,odds,Odds (Decimal),sport_toto
0,Jannik Sinner vs Taylor Fritz,Wedstrijd,Wedstrijdnotering,"Sinner, Jannik",NaN,1150.0,1.15,Tennis
1,Jannik Sinner vs Taylor Fritz,Wedstrijd,Wedstrijdnotering,"Fritz, Taylor",NaN,5800.0,6.00,Tennis
2,Carlos Alcaraz vs Andrey Rublev,Wedstrijd,Wedstrijdnotering,"Alcaraz, Carlos",NaN,1340.0,1.30,Tennis
3,Carlos Alcaraz vs Andrey Rublev,Wedstrijd,Wedstrijdnotering,"Rublev, Andrey",NaN,3350.0,3.65,Tennis
4,Alexander Zverev vs Casper Ruud,Wedstrijd,Wedstrijdnotering,"Zverev, Alexander",NaN,1140.0,1.15,Tennis
...,...,...,...,...,...,...,...,...
236,Mark Selby vs Shaun Murphy,Wedstrijd,Frame 1,"Murphy, Shaun",NaN,1910.0,2.10,Snooker
238,Mark Selby vs Shaun Murphy,Wedstrijd,Meeste Century Breaks,"Selby, Mark",NaN,3000.0,1.73,Snooker
240,Mark Selby vs Shaun Murphy,Wedstrijd,Meeste Century Breaks,"Murphy, Shaun",NaN,3250.0,2.10,Snooker
242,Mark Selby vs Shaun Murphy,Wedstrijd,Meeste Half-Century Breaks,"Selby, Mark",NaN,2000.0,1.73,Snooker
